In [1]:
from fifo import AxiSFIFO
import numpy as np

**Test 1**: In an environment where the sink has 80% chance to receive the current data

In [2]:
fifo = AxiSFIFO("../../build/exec/fifo4")

test_length = 8192

input_data  = np.random.randint(0, 256, test_length)
input_tlast  = np.random.randint(0, 10, test_length) < 1 # 10 % tlast signal
output_data = [] 
output_tlast = [] 

send_count = 0
recv_count = 0

try: 
    fifo.forward(rst_n=False)
    fifo.backward(True, 0, False, True)

    while recv_count < test_length:
        if send_count < test_length:
            data    = input_data[send_count]
            tlast   = input_tlast[send_count]
            valid   = True
        else:
            data    = 0
            tlast   = False
            valid   = False
            
        output = fifo.forward()
        nxt_ready = (np.random.randint(0, 10) > 1) # 80% ready
        if nxt_ready and output["valid"]:
            output_data.append(output["data"])
            output_tlast.append(output["last"])
            recv_count += 1

        cur_ready = fifo.backward(valid, data, nxt_ready, last=tlast)
        if cur_ready:
            send_count = send_count + 1
finally:
    fifo.terminate()
del(fifo)

output_data    = np.array(output_data)
output_tlast    = np.array(output_tlast)

data_assertion = np.all(input_data == output_data)
tlast_assertion = np.all(input_tlast == output_tlast)

print("All data  received correctly: {}".format(data_assertion))
print("All tlast received correctly: {}".format(tlast_assertion))

All data  received correctly: True
All tlast received correctly: True


In [4]:
fifo = AxiSFIFO("../../build/exec/fifo4")

test_length = 8192

input_data  = np.random.randint(0, 256, test_length)
input_tlast  = np.random.randint(0, 10, test_length) < 1 # 10 % tlast signal
input_valid = np.random.randint(0, 10, test_length) > 3 # 60 % valid signal
input_ready = np.random.randint(0, 10, test_length) > 3 # 60 % ready signal
output_data  = np.empty_like(input_data)
output_tlast  = np.empty_like(input_tlast)
output_valid = np.empty_like(input_tlast)
output_ready = np.empty_like(input_tlast)

send_count = 0

try:
    fifo.forward(rst_n=False)
    fifo.backward(True, 33, False)

    for i in range(test_length):
        data      = input_data[i]
        tlast      = input_tlast[i]
        valid     = input_valid[i]
        nxt_ready = input_ready[i]

        output          = fifo.forward()
        output_data[i]  = output["data"]
        output_tlast[i]  = output["last"]
        output_valid[i] = output["valid"]
    
        cur_ready = fifo.backward(valid, data, nxt_ready, tlast)
        output_ready[i] = cur_ready
finally:
    fifo.terminate()

del(fifo)

received_data = input_data[input_valid * output_ready]
received_tlast = input_tlast[input_valid * output_ready]

produced_data = output_data[output_valid * input_ready]
produced_tlast = output_tlast[output_valid * input_ready]

data_assertion = np.all(received_data[:len(produced_data)] == produced_data)
tlast_assertion = np.all(received_tlast[:len(produced_data)] == produced_tlast)

print("All data  received by the FIFO is finally produced: {}".format(data_assertion))
print("All tlast received by the FIFO is finally produced: {}".format(tlast_assertion))


All data  received by the FIFO is finally produced: True
All tlast received by the FIFO is finally produced: True
